In [43]:
import pandas
import numpy as np
import sklearn.preprocessing as preprocessing
import matplotlib.pyplot as plt
import nltk
import scipy
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, LSTM, Input
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
import pickle
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_validate, GridSearchCV
import sklearn.metrics as metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from nltk.probability import FreqDist
from sklearn.metrics import classification_report

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ishan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ishan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data Preprocessing / EDA

In [2]:
df = pandas.read_csv(r"C:\Users\ishan\Desktop\Project\DJIA-prediction-from-news\Combined_News_DJIA.csv")

df.isna().sum()

Date     0
Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    1
Top24    3
Top25    3
dtype: int64

In [3]:
df.fillna("0",inplace=True)
df.shape

(1989, 27)

In [50]:
df['Label'].head(5)

0    0
1    1
2    0
3    0
4    1
Name: Label, dtype: int64

In [4]:
df_train=df[df['Date']<'20150101']
df_test=df[df['Date']>'20141231']

In [10]:
df_train.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."


## NLP Pipeline

In [5]:
#Remove all chars except the letters

def clean_data(dataset):
    
    #ignore DateTime and Label
    data = dataset.iloc[:,2:27]
    data.replace("[^a-zA-Z]", " ", regex=True, inplace=True)
    return data

def combine_data(data):
    
    #Merge into one column
    headlines = []
    for i in range(len(data.index)):
        headlines.append(' '.join(str(x) for x in data.iloc[i, :]))
    return headlines

# Another possible method: clean,combine -> Tokenize -> Stemm -> embedding (tensorflow)


def lemmatize_data(data, lemmatizer):
    
    cleaned_dataset = []
    for i in range(len(data)):
        clean_text = data[i].lower()
        clean_text = clean_text.split()
        clean_text = [lemmatizer.lemmatize(word) for word in clean_text if word not in stopwords.words('english')]
        cleaned_dataset.append(' '.join(clean_text))
    return cleaned_dataset

# Returns sparse matrix

def vectorize_data(data, cv):
    
    vectorized_dataset = cv.fit_transform(data)
    return vectorized_dataset



In [6]:
cleaned_train = clean_data(df_train)
cleaned_test = clean_data(df_test)

In [11]:
cleaned_train.head(2)

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,b Georgia downs two Russian warplanes as cou...,b BREAKING Musharraf to be impeached,b Russia Today Columns of troops roll into So...,b Russian tanks are moving towards the capital...,b Afghan children raped with impunity U N ...,b Russian tanks have entered South Ossetia...,b Breaking Georgia invades South Ossetia Rus...,b The enemy combatent trials are nothing but...,b Georgian troops retreat from S Osettain cap...,b Did the U S Prep Georgia for War with Russia,...,b Georgia Invades South Ossetia if Russia ge...,b Al Qaeda Faces Islamist Backlash,b Condoleezza Rice The US would not act to p...,b This is a busy day The European Union has ...,b Georgia will withdraw soldiers from Ir...,b Why the Pentagon Thinks Attacking Iran is a ...,b Caucasus in crisis Georgia invades South Os...,b Indian shoe manufactory And again in a se...,b Visitors Suffering from Mental Illnesses Ban...,b No Help for Mexico s Kidnapping Surge
1,b Why wont America and Nato help us If they w...,b Bush puts foot down on Georgian conflict,b Jewish Georgian minister Thanks to Israeli ...,b Georgian army flees in disarray as Russians ...,b Olympic opening ceremony fireworks faked,b What were the Mossad with fraudulent New Zea...,b Russia angered by Israeli military sale to G...,b An American citizen living in S Ossetia blam...,b Welcome To World War IV Now In High Definit...,b Georgia s move a mistake of monumental prop...,...,b Israel and the US behind the Georgian aggres...,b Do not believe TV neither Russian nor Geor...,b Riots are still going on in Montreal Canada...,b China to overtake US as largest manufacturer,b War in South Ossetia PICS,b Israeli Physicians Group Condemns State Tort...,b Russia has just beaten the United States ov...,b Perhaps the question about the Georgia R...,b Russia is so much better at war,b So this is what it s come to trading sex fo...


In [12]:
combined_train = combine_data(cleaned_train)
combined_test = combine_data(cleaned_test)

In [34]:
print(combined_train[:1])
print('\n Number of words: ',len(combined_train[:1][0]))

['b Georgia  downs two Russian warplanes  as countries move to brink of war  b BREAKING  Musharraf to be impeached   b Russia Today  Columns of troops roll into South Ossetia  footage from fighting  YouTube   b Russian tanks are moving towards the capital of South Ossetia  which has reportedly been completely destroyed by Georgian artillery fire  b Afghan children raped with  impunity   U N  official says   this is sick  a three year old was raped and they do nothing  b     Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets   b Breaking  Georgia invades South Ossetia  Russia warned it would intervene on SO s side  b The  enemy combatent  trials are nothing but a sham  Salim Haman has been sentenced to       years  but will be kept longer anyway just because they feel like it   b Georgian troops retreat from S  Osettain capital  presumably leaving several hundred people killed   VIDEO   b Did the U S  Prep Georgia for War with Russia   b Rice Gives Gree

In [15]:
lemmatizer = WordNetLemmatizer()

lemm_train = lemmatize_data(combined_train, lemmatizer)
lemm_test = lemmatize_data(combined_test, lemmatizer)

In [35]:
print(lemm_train[:1])
print('\n Number of words: ',len(lemm_train[:1][0]))

['b georgia down two russian warplane country move brink war b breaking musharraf impeached b russia today column troop roll south ossetia footage fighting youtube b russian tank moving towards capital south ossetia reportedly completely destroyed georgian artillery fire b afghan child raped impunity u n official say sick three year old raped nothing b russian tank entered south ossetia whilst georgia shoot two russian jet b breaking georgia invades south ossetia russia warned would intervene side b enemy combatent trial nothing sham salim haman sentenced year kept longer anyway feel like b georgian troop retreat osettain capital presumably leaving several hundred people killed video b u prep georgia war russia b rice give green light israel attack iran say u veto israeli military ops b announcing class action lawsuit behalf american public fbi b russia georgia war nyt top story opening ceremony olympics fucking disgrace yet proof decline journalism b china tell bush stay country affai

In [17]:
cv = CountVectorizer(ngram_range=(2,2))

vec_data_train = vectorize_data(lemm_train,cv)
vec_data_test = cv.transform(lemm_test)

In [36]:
vec_data_train.shape

(1863, 393617)

## ML 

In [45]:
y_train=df_train['Label']
X_train=vec_data_train
y_test= df_test['Label']

### XG Boost

In [ ]:

param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['binary:logistic']
}

xgb_model = XGBClassifier()

gsearch = GridSearchCV(estimator = xgb_model, param_grid = param_tuning,cv = 5,n_jobs = -1,verbose = 1)

gsearch.fit(X_train,y_train)

gsearch.best_params_




In [41]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=42)

xgb1.fit(X_train,y_train)

[19:17:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=4, nthread=4, num_parallel_tree=1,
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=42, subsample=0.8, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [42]:
y_pred = xgb1.predict(vec_data_test)
predictions = [round(value) for value in y_pred]

In [46]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.82      0.83      0.83       186
           1       0.84      0.82      0.83       192

    accuracy                           0.83       378
   macro avg       0.83      0.83      0.83       378
weighted avg       0.83      0.83      0.83       378



In [52]:
cm = confusion_matrix(y_test, predictions)
tn, fp, fn, tp = cm.ravel()
(tn, fp, fn, tp)

recall = tp/(tp+fn)
print('Recall:',recall)
precision = tp/(fp+tp)
print('Precision',precision)

Recall: 0.8229166666666666
Precision 0.8359788359788359


### Random Forest 

In [47]:
clf_rf = RandomForestClassifier(random_state=42).fit(X_train,y_train)


rf_pred = clf_rf.predict(vec_data_test)

In [48]:
print(classification_report(y_test,rf_pred))

              precision    recall  f1-score   support

           0       1.00      0.70      0.82       186
           1       0.77      1.00      0.87       192

    accuracy                           0.85       378
   macro avg       0.89      0.85      0.85       378
weighted avg       0.89      0.85      0.85       378



In [53]:
cm = confusion_matrix(y_test, rf_pred)
tn, fp, fn, tp = cm.ravel()
(tn, fp, fn, tp)

recall = tp/(tp+fn)
print('Recall:',recall)
precision = tp/(fp+tp)
print('Precision',precision)

Recall: 1.0
Precision 0.7741935483870968


### 2-Layered Perceptron

In [133]:
clf_mlp = MLPClassifier(hidden_layer_sizes=(100,2),activation='relu',random_state=42, max_iter=100)

clf_mlp.fit(X_train, y_train)

mlp_pred =clf_mlp.predict(vec_data_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [134]:
print(classification_report(y_test,mlp_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       186
           1       0.51      1.00      0.67       192

    accuracy                           0.51       378
   macro avg       0.25      0.50      0.34       378
weighted avg       0.26      0.51      0.34       378



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [135]:
cm = confusion_matrix(y_test, mlp_pred)
tn, fp, fn, tp = cm.ravel()
(tn, fp, fn, tp)

recall = tp/(tp+fn)
print('Recall:',recall)
precision = tp/(fp+tp)
print('Precision',precision)

Recall: 1.0
Precision 0.5079365079365079


In [ ]:
#recall = 64%

# precision = 100% 

## Ensemble

In [137]:

def custom_ensemble(clf_mlp , clf_rf , mlp_weight , rf_weight ):
    
    mlp_pred_proba = clf_mlp.predict_proba(vec_data_test)

    fr_pred_proba = clf_rf.predict_proba(vec_data_test)
    
    idx = df_test['Label'].index
    mlp = pandas.DataFrame(mlp_pred_proba,index=idx)
    rf= pandas.DataFrame(fr_pred_proba,index=idx)

    combined = pandas.concat([rf[0],mlp[1]],axis=1)

    rf2 = pandas.DataFrame(np.array([rf.iloc[i,0]>0.5 for i in range(len(rf))]).astype(int),index=idx)
    mlp2 = pandas.DataFrame(np.array([mlp.iloc[i,1]>0.5 for i in range(len(mlp))]).astype(int),index=idx)

    average_prob = pandas.concat([rf[0]*rf_weight,mlp[1]*mlp_weight],axis=1)
    
    ensemble_pred = np.array([average_prob.iloc[i,1]>average_prob.iloc[i,0] for i in range(len(average_prob))]).astype(int)

    return ensemble_pred


In [143]:
ensemble_pred=custom_ensemble(clf_mlp , clf_rf , mlp_weight=1 , rf_weight=0.8)

In [144]:
print(classification_report(y_test, ensemble_pred))

              precision    recall  f1-score   support

           0       1.00      0.27      0.43       186
           1       0.59      1.00      0.74       192

    accuracy                           0.64       378
   macro avg       0.79      0.64      0.59       378
weighted avg       0.79      0.64      0.59       378



In [145]:
cm = confusion_matrix(y_test,ensamble_pred)
tn, fp, fn, tp = cm.ravel()
(tn, fp, fn, tp)

recall = tp/(tp+fn)
print('Recall:',recall)
precision = tp/(fp+tp)
print('Precision',precision)

Recall: 1.0
Precision 0.5079365079365079


In [ ]:
# recall = 80%

# precision = 95%